In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using Theano backend.


In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [3]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

# free some space
del train 

In [4]:
# Normalize the data
X_train = X_train / 255.0
test = test / 255.0
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,1,28,28)
test = test.values.reshape(-1,1,28,28)

In [5]:
Y_train = to_categorical(Y_train, num_classes = 10)

# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)
print(X_train.shape, Y_train.shape)

(37800, 1, 28, 28) (37800, 10)


In [6]:
model = Sequential()

# conv1
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same', 
                 activation ='relu', input_shape = (1,28,28)))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# conv2
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same', 
                 activation ='relu', input_shape = (1,28,28)))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# conv3
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.3))

# flatten
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(10, activation = "softmax"))
# Compile the model

opt = Adam(lr=0.001)
model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
print("compile done")

compile done


In [7]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

/home/jac/.local/lib/python3.5/site-packages/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_first" (channels on axis 1), i.e. expected either 1, 3 or 4 channels on axis 1. However, it was passed an array with shape (37800, 1, 28, 28) (1 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')


In [9]:
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.models import load_model

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.6, 
                                            min_lr=0.00001)
#2017-09-05 log
#275s - loss: 0.0475 - acc: 0.9847 - val_loss: 0.0183 - val_acc: 0.9945 64 filter version
#Epoch 30/60
    
best_model_path = 'best_m_v2.h5'
best_model = ModelCheckpoint(filepath=best_model_path, monitor='acc', save_best_only=True)
epochs = 100
steps_per_epoch = 100
batch_size = 64
model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=steps_per_epoch,
                              callbacks=[best_model,learning_rate_reduction])

Epoch 1/100
58s - loss: 1.6076 - acc: 0.4256 - val_loss: 0.4876 - val_acc: 0.8560
Epoch 2/100
58s - loss: 0.6638 - acc: 0.7847 - val_loss: 0.1906 - val_acc: 0.9429
Epoch 3/100
58s - loss: 0.4340 - acc: 0.8639 - val_loss: 0.1296 - val_acc: 0.9593
Epoch 4/100
59s - loss: 0.3329 - acc: 0.8969 - val_loss: 0.1630 - val_acc: 0.9512
Epoch 5/100
64s - loss: 0.2711 - acc: 0.9195 - val_loss: 0.1005 - val_acc: 0.9705
Epoch 6/100
63s - loss: 0.2479 - acc: 0.9307 - val_loss: 0.0817 - val_acc: 0.9745
Epoch 7/100
61s - loss: 0.2215 - acc: 0.9300 - val_loss: 0.0768 - val_acc: 0.9764
Epoch 8/100
60s - loss: 0.1839 - acc: 0.9450 - val_loss: 0.0742 - val_acc: 0.9786
Epoch 9/100
63s - loss: 0.1922 - acc: 0.9408 - val_loss: 0.0874 - val_acc: 0.9745
Epoch 10/100
60s - loss: 0.1882 - acc: 0.9452 - val_loss: 0.0639 - val_acc: 0.9800
Epoch 11/100
58s - loss: 0.1726 - acc: 0.9508 - val_loss: 0.0664 - val_acc: 0.9802
Epoch 12/100
60s - loss: 0.1535 - acc: 0.9573 - val_loss: 0.0547 - val_acc: 0.9833
Epoch 13/100


KeyboardInterrupt: 

In [10]:
# predict results use last model
results = model.predict(test)
# select the indix with the maximum probability
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
import arrow
ts = str(arrow.now())[:-10].replace(':','_').replace('.',"_")
res_file = "play_cnn_mnist_datagen_{}.csv".format(ts)
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv(res_file,index=False)
print('save done',res_file)

save done play_cnn_mnist_datagen_2017-09-06T11_28_26_29.csv


In [11]:
model = load_model(best_model_path)  # try callback model
# predict results
results = model.predict(test)
# select the indix with the maximum probability
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
ts = str(arrow.now())[:-10].replace(':','_').replace('.',"_")
res_file = "play_cnn_mnist_datagen_{}.csv".format(ts)
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv(res_file,index=False)
print('save done',res_file)

save done play_cnn_mnist_datagen_2017-09-06T11_29_44_87.csv
